# Парсинг новостной ленты

Для парсинга я использовал сайт финансовых новостей finam. Особенность сайта в том, что сами новости подгружаются на нем динамически и по прямой загрузке станицы их не видно.

Решено использовать selenium для парсинга, так как API по условиям задачи использовать не рекомендовано. Ниже простой код для парсинга небольшого блока новостей. Извлекаются заголовок и текст новости.

In [1]:
# preload local libs
import sys
import os
import pathlib

project_dir = str(pathlib.Path(os.getcwd()).parent.resolve())
if project_dir not in sys.path:
    sys.path.insert(1, project_dir)

# project_dir

In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

from lib import utils

import warnings

warnings.filterwarnings("ignore")

In [32]:
link = 'https://www.finam.ru/publications/section/companies/'

In [28]:
driver_path = '../chromedriver-linux64/chromedriver'

In [26]:
options = webdriver.ChromeOptions()

# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--whitelisted-ips=')

In [66]:
driver = webdriver.Chrome(service=webdriver.chrome.service.Service(driver_path), options=options)
driver.set_window_size(1440, 900)
driver.get(link)

news = []

# Загрузка контента происходит динамически на странице, поэтому ждем некоторое время до появления
try:
    dynamic_div = WebDriverWait(driver, 3).until(
        EC.visibility_of_element_located((By.ID, "infinity-ui-main"))  # или By.CLASS_NAME, By.XPATH и т.д.
    )
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # основной контейнер
    div_content = soup.find('div', id='infinity-ui-main')

    for one_news in div_content.select('div.mb2x'):
        title = one_news.div.select('div')[1].text.strip()
        text = one_news.div.p.text.strip()
        news.append((title, text))
finally:
    driver.quit()

In [68]:
news[:3]

[('В России введут регулярный мониторинг финансового положения пенсионеров',
  'В результате будет проведен "комплексный анализ социально-экономического положения старшего поколения'),
 ('Chevron и Quantum Energy Partners объединили усилия для покупки активов "ЛУКОЙЛа"',
  'Если сделка состоится, Chevron и Quantum планируют разделить между собой активы, которые, по оценке "ЛУКОЙЛа", составляют $22 млрд'),
 ('Торги на СПБ Бирже проходят спокойно - инвесторы покупают "Полюс", продают "Т-Техно"',
  'Торги на Московской бирже не проводятся по случаю празднования Рождества')]